In [1]:
%load_ext cython

In [2]:
from galois.shmem import *
from galois.graphs import *

In [3]:
print(setActiveThreads(16))

16


## Load some graph data

In [4]:
g = LC_CSR_Graph_Directed_primitive["uint32_t", "void"]() 
# g.readGraphFromGRFile("../katana-build/inputs/reference/structured/rome99.cgr")
g.readGraphFromGRFile("/home/amp/Downloads/indochina-2004.csgr")
print(g, len(g))

<galois.graphs.LC_CSR_Graph_Directed_uint32_t_void object at 0x7f31d01da0c0> 7414866


## An inline implementation of BFS

In [5]:
%%cython
# distutils: extra_link_args=["-lgalois_shmem"]

from galois.graphs cimport *

ctypedef LC_CSR_Graph[uint32_t, void, dummy_true] Graph_CSR
ctypedef LC_CSR_Graph[uint32_t, void, dummy_true].GraphNode GNodeCSR


cdef void operator(Graph_CSR *g, GNodeCSR n) nogil:
    d = &g.getData(n)
    d[0] = g.size()


def initialize_for_bfs(LC_CSR_Graph_Directed_uint32_t_void graph):
    g = &graph.underlying
    with nogil:
        do_all(iterate(g[0]), bind_leading(operator, g))
        
#
# BFS sync operator to be executed on each Graph node
#
cdef void bfs_sync_operator(Graph_CSR *g, InsertBag[GNodeCSR] *next, int nextLevel, GNodeCSR n) nogil:
    numNodes = g[0].size()
    src_data = &g[0].getData(n)
    edges = g.edges(n)
    for ii in edges:
        dst = g.getEdgeDst(ii)
        dst_data = &g.getData(dst)
        if dst_data[0] == numNodes:
            dst_data[0] = nextLevel
            next.push(dst)

def bfs_sync(LC_CSR_Graph_Directed_uint32_t_void graph, int source):
    cdef:
        Timer T
        InsertBag[GNodeCSR] curr, next
        uint32_t nextLevel = 0;
    next.push(<GNodeCSR>source)
    graph.setData(source, 0)
    T.start()
    with nogil:
        while not next.empty():
            curr.swap(next)
            next.clear()
            nextLevel += 1
            do_all(iterate(curr),
                     bind_leading(&bfs_sync_operator, &graph.underlying, &next, nextLevel), no_pushes(), steal(),
                     loopname("bfs_sync"))
    T.stop()

In [6]:
initialize_for_bfs(g)
bfs_sync(g, 0)

In [7]:
print([g.getData(i) for i in range(20)])

[0, 1, 1, 1, 1, 1, 2, 1, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
